保存当前conda环境的命令是：`conda env export > environment.yml`

需要下载nltk数据的话取消注释并执行：

In [1]:
import nltk
nltk.set_proxy('http://localhost:17891')  # 根据自己的代理设置修改
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HenryFox\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HenryFox\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HenryFox\AppData\Roaming\nltk_data...


True

如果要训练的话，执行下面的代码（来自`training_py.py`），会得到`chatbotmodel.h5`、`words.pkl`、`classes.pkl`文件：

In [2]:
import random
import json
import pickle
import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer=WordNetLemmatizer()

with open('intents.json') as json_file:
    intents = json.load(json_file)

#print(intents)

words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


words =[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



training=[]
output_empty=[0]*len(classes)

for document in documents:
  bag=[]
  word_patterns=document[0]
  words = [lemmatizer.lemmatize(word) for word in words if word and word not in ignore_letters]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row=list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag,output_row])

random.shuffle(training)
train_x=np.array([x[0] for x in training])
train_y=np.array([x[1] for x in training])

train_x=list(train_x)
train_y=list(train_y)
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbotmodel.h5', hist)
print('Training Done')

Epoch 1/200
17/17 [==============================] - 1s 3ms/step - loss: 2.7398 - accuracy: 0.0476 
Epoch 2/200
17/17 [==============================] - 0s 3ms/step - loss: 2.6994 - accuracy: 0.1071
Epoch 3/200
17/17 [==============================] - 0s 3ms/step - loss: 2.6567 - accuracy: 0.1667
Epoch 4/200
17/17 [==============================] - 0s 3ms/step - loss: 2.5620 - accuracy: 0.1786
Epoch 5/200
17/17 [==============================] - 0s 3ms/step - loss: 2.4665 - accuracy: 0.2381
Epoch 6/200
17/17 [==============================] - 0s 3ms/step - loss: 2.3716 - accuracy: 0.2262
Epoch 7/200
17/17 [==============================] - 0s 3ms/step - loss: 2.3684 - accuracy: 0.2024
Epoch 8/200
17/17 [==============================] - 0s 3ms/step - loss: 2.2097 - accuracy: 0.3333
Epoch 9/200
17/17 [==============================] - 0s 3ms/step - loss: 2.1394 - accuracy: 0.2857
Epoch 10/200
17/17 [==============================] - 0s 3ms/step - loss: 2.0907 - accuracy: 0.2857
Epoch 11

如果要运行，执行以下代码（来自`chatbot_py.py`）：

In [ ]:
# -*- coding: utf-8 -*-
"""chatbot.py

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1feCkdIYr8e1V5KIzxxooJr5o2bRK4R4p
"""

import nltk
import random
import numpy as np
import json
import pickle
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
lemmatizer=WordNetLemmatizer()

with open('intents.json') as json_file:
    intents = json.load(json_file)

words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent': classes[r[0]],'probability':str(r[1])})
  return return_list

def get_response(intents_list,intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

print("GO! BOT IS RUNNING")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

GO! BOT IS RUNNING


 How old are you


1/1 [==============================] - 0s 74ms/step
I get reborn after every compilation


GUI界面：执行`python gui.py`即可